Market depth (order book)
==============

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=16)

<IB connected to 127.0.0.1:7497 clientId=16>

To get a list of all exchanges that support market depth data and display the first five:

In [2]:
l = ib.reqMktDepthExchanges()
l[:5]

[DepthMktDataDescription(exchange='DTB', secType='OPT', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='LSEETF', secType='STK', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='SGX', secType='FUT', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='IDEALPRO', secType='CASH', serviceDataType='Deep', aggGroup=4),
 DepthMktDataDescription(exchange='ARCA', secType='STK', serviceDataType='Deep')]

Let's subscribe to market depth data for EURUSD:

In [3]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)
ticker = ib.reqMktDepth(contract)

To see a live order book, an event handler for ticker updates is made that displays a dynamically updated dataframe:

In [4]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5),
        columns='bidSize bidPrice askPrice askSize'.split())

def onTickerUpdate(ticker):
    l = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = l[i].size if i < len(l) else 0
        df.iloc[i, 1] = l[i].price if i < len(l) else 0
    l = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = l[i].price if i < len(l) else 0
        df.iloc[i, 3] = l[i].size if i < len(l) else 0
    clear_output(wait=True)
    display(df)

ticker.updateEvent += onTickerUpdate

IB.sleep(15)

,bidSize,bidPrice,askPrice,askSize
0,9500000,1.18736,1.18739,1000000
1,10000000,1.18735,1.1874,5500000
2,12000000,1.18734,1.18741,11500000
3,7000000,1.18733,1.18742,2500000
4,156456,1.187,1.18743,1500000


Stop the market depth subscription:

In [5]:
ib.cancelMktDepth(contract)

In [6]:
ib.disconnect()